In [71]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import csv

DRIVER_PATH = "./chromedriver"
driver = webdriver.Chrome(DRIVER_PATH)


/var/folders/dw/tbc1pbpn00s51dcj9nf620n40000gn/T/ipykernel_38759/860795275.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


In [72]:
def get_link(driver, link):
    driver.get(link)
    sleep(2)
    try:
        driver.find_element(By.XPATH, '//*[@id="didomi-popup"]/div/div/div/span').click()
    except:
        pass
    return driver


In [73]:
def get_manga_information(driver):
    # senariste, dessinateur
    balise_xpath = {
        'title': '//*[@id="content"]/div[1]/div[2]/h1/span',
        'informations': '//*[@id="onglets_1_information"]/div[1]/div[2]/ul/li',
        'description': '//*[@id="onglets_1_information"]/div[2]/div/div[1]',
        'Links': '//*[@id="edition_0-1"]/div/a'
        }
    informations = {'Links': []}
    for key, xpath in balise_xpath.items():
        if key == 'Links':
            elems = driver.find_elements(By.XPATH, xpath)
            for elem in elems:
                informations['Links'].append(elem.get_attribute("href"))
            continue
        if key == 'informations':
            elems = driver.find_elements(By.XPATH, xpath)
            for elem in elems:
                splited = elem.text.split(' : ')
                if len(splited) == 1:
                    continue
                if splited[0] == 'Auteur':
                    informations['Scénariste'] = splited[1]
                    informations['Dessinateur'] = splited[1]
                    continue
                informations[splited[0]] = splited[1]
        try:
            informations[key] = driver.find_element(By.XPATH, xpath).text.replace('\n', ' ').replace('\r', '')
        except:
            print(xpath)
    return informations

In [74]:
def push_to_csv(manga):
    row_push = [
        manga['title'] if 'title' in manga else '',
        manga['description'] if 'description' in manga else '',
        manga['Origine'] if 'Origine' in manga else '',
        manga['Type'] if 'Type' in manga else '',
        manga['Genres'] if 'Genres' in manga else '',
        manga['Thèmes'] if 'Thèmes' in manga else '',
        manga['Scénariste'] if 'Scénariste' in manga else '',
        manga['Dessinateur'] if 'Dessinateur' in manga else ''
    ]

    with open('./manga_information_result.csv', 'a+', newline='') as csv_result_file:
        csv_writer = csv.writer(csv_result_file, delimiter=';')
        csv_writer.writerow(row_push)


In [79]:
def main(driver):
    prepare_file()
    links = []
    for i in range(4):
        base_link = f'https://www.nautiljon.com/mangas/?dbt={i*50}'
        driver = get_link(driver, base_link)
        mangas = driver.find_elements(By.XPATH, '//*[@id="content"]/div[1]/div[5]/table/tbody/tr')
        for manga in mangas:
            links.append(manga.find_element(By.XPATH, 'td[2]/a').get_attribute('href'))
        sleep(2)
        
    for link in links:
        driver.get(link)
        manga = get_manga_information(driver)
        do_tomes(driver, manga['Links'])
        push_to_csv(manga)
        sleep(1)
    

In [76]:
def do_tomes(driver, links):
    for link in links:
        driver = get_link(driver, link)
        try:
            tome_title=driver.find_element(By.CLASS_NAME, "h1titre").text.split('\n')[1]
        except:
            tome_title=''
        try:
            tome_mark=driver.find_element(By.CLASS_NAME, "moy_note").text
        except:
            tome_mark=''
            
        try:
            infos_list=driver.find_element(By.CLASS_NAME, "mb10").find_elements(By.TAG_NAME, "li")
        except:
            return
        
        try:
            tome_publisher=infos_list[0].text.split(':')[1]
        except:
            tome_publisher=''
            
        try:
            tome_published_date=infos_list[1].text.split(':')[1]
        except:
            tome_published_date=''
            
        try:
            tome_number_pages=infos_list[3].text.split(':')[1]
        except:
            tome_number_pages=''
            
        try:
            tome_description=driver.find_element(By.CLASS_NAME, "description").text.replace('\n', ' ').replace('\r', '')
        except:
            tome_description=''
        
        row_push = [tome_title,tome_mark, tome_description, tome_publisher, tome_published_date, tome_number_pages]
        with open('./tome_information_result.csv', 'a+', newline='') as csv_result_file:
            csv_writer = csv.writer(csv_result_file, delimiter=';')
            csv_writer.writerow(row_push)

            

In [77]:
def prepare_file():
    
    try:
        with open('./manga_information_result.csv', 'r') as file:
            csvreader = csv.reader(file)
            row_count = sum(1 for row in csvreader)
    except FileNotFoundError:
        row_push = ['Title','Description','Origine','Type','Genres','Thèmes','Scénariste','Dessinateur']
        with open('./manga_information_result.csv', 'w', newline='') as csv_result_file:
            csv_writer = csv.writer(csv_result_file, delimiter=';')
            csv_writer.writerow(row_push)
    
    try:
        with open('./tome_information_result.csv', 'r') as file:
            csvreader = csv.reader(file)
            row_count = sum(1 for row in csvreader)
    except FileNotFoundError:
        row_push = ['Title','Note','Description','Éditeur','Date de publication','Nombre de pages']
        with open('./tome_information_result.csv', 'w', newline='') as csv_result_file:
            csv_writer = csv.writer(csv_result_file, delimiter=';')
            csv_writer.writerow(row_push)

In [78]:
main(driver)